### 1º Juntar as tabelas ###


In [17]:
#import pandas as pd
#%pip install pyspark
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder \
    .master("local") \
    .appName("BigData") \
    .getOrCreate()



In [ ]:
spark.stop()

In [3]:
# tabela1 = pd.read_csv('job_skills.csv')

# print(tabela1.head())

tabela1 = spark.read.csv('job_skills.csv', header=True) 
tabela1.show(5)


+--------------------+--------------------+
|            job_link|          job_skills|
+--------------------+--------------------+
|https://www.linke...|Building Custodia...|
|https://www.linke...|Customer service,...|
|https://www.linke...|Applied Behavior ...|
|https://www.linke...|Electrical Engine...|
|https://www.linke...|Electrical Assemb...|
+--------------------+--------------------+
only showing top 5 rows



In [4]:
tabela1.schema

StructType([StructField('job_link', StringType(), True), StructField('job_skills', StringType(), True)])

In [5]:
# tabela2 = pd.read_csv('job_summary.csv')
# print(tabela2.head())

tabela2 = spark.read.csv('job_summary.csv', header=True)
tabela2.show(5)

+--------------------+--------------------+
|            job_link|         job_summary|
+--------------------+--------------------+
|https://www.linke...|Rock N Roll Sushi...|
|As our Restaurant...| you’ll never be ...|
|            We Offer|                NULL|
|Competitive compe...|                NULL|
|  Insurance benefits|                NULL|
+--------------------+--------------------+
only showing top 5 rows



In [6]:
tabela2.schema

StructType([StructField('job_link', StringType(), True), StructField('job_summary', StringType(), True)])

In [7]:
# tabela3 = pd.read_csv('linkedin_job_postings.csv')
# print(tabela3.head())

tabela3 = spark.read.csv('linkedin_job_postings.csv', header=True)
tabela3.show(5)

+--------------------+--------------------+-----------+-------+---------------+--------------------+--------------------+--------------------+----------+-----------+--------------+--------------------+----------+--------+
|            job_link| last_processed_time|got_summary|got_ner|is_being_worked|           job_title|             company|        job_location|first_seen|search_city|search_country|     search_position| job_level|job_type|
+--------------------+--------------------+-----------+-------+---------------+--------------------+--------------------+--------------------+----------+-----------+--------------+--------------------+----------+--------+
|https://www.linke...|2024-01-21 07:12:...|          t|      t|              f|Account Executive...|                  BD|       San Diego, CA|2024-01-15|   Coronado| United States|         Color Maker|Mid senior|  Onsite|
|https://www.linke...|2024-01-21 07:39:...|          t|      t|              f|Registered Nurse ...|   Trinity H

In [8]:
tabela3.schema

StructType([StructField('job_link', StringType(), True), StructField('last_processed_time', StringType(), True), StructField('got_summary', StringType(), True), StructField('got_ner', StringType(), True), StructField('is_being_worked', StringType(), True), StructField('job_title', StringType(), True), StructField('company', StringType(), True), StructField('job_location', StringType(), True), StructField('first_seen', StringType(), True), StructField('search_city', StringType(), True), StructField('search_country', StringType(), True), StructField('search_position', StringType(), True), StructField('job_level', StringType(), True), StructField('job_type', StringType(), True)])

In [9]:
# df = pd.merge(tabela1, tabela2, on='job_link', how='inner')
# df = pd.merge(df, tabela3, on='job_link', how='inner')
# print(df)

# df = tabela1.join(tabela2, on='job_link', how='inner').join(tabela3, on='job_link', how='inner')
# df.show()

#inner join que era suposto fazer, mas alteração de ideias 



### 2º Limpeza dos dados por tabela ###

In [16]:
spark.stop()

Tabela_1

In [10]:
tabela1.count()

1296381

In [11]:
from pyspark.sql.functions import col
print( tabela1.filter(col("job_link").isNull()).count())
print(tabela1.filter(col("job_skills").isNull()).count())

0
2007


In [12]:
tabela_limpa = tabela1.dropna(subset=["job_link", "job_skills"])

In [13]:
tabela_limpa.count()

1294374

In [16]:
tabela_limpa = tabela_limpa.dropDuplicates()
num_linhas_depois = tabela_limpa.count()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
tabela_limpa.write.csv("job_skills_clean.csv", header=True)

Tabela_2

In [ ]:
from pyspark.sql.functions import col, sum as spark_sum
tabela2.select([spark_sum(col(c).isNull().cast('int')).alias(c) for c in tabela2.columns]).show()

In [ ]:
tabela2 = tabela2.dropna()
tabela2.select([spark_sum(col(c).isNull().cast('int')).alias(c) for c in tabela2.columns]).show()

In [ ]:
print(num_linhas_antes = tabela2.count())
tabela_2 = tabela2.dropDuplicates()
print(num_linhas_depois = tabela_2.count())

In [ ]:
tabela_2.write.csv("job_summary_clean.csv", header=True)

Tabela_3

In [ ]:
from pyspark.sql.functions import col, sum as spark_sum
tabela3.select([spark_sum(col(c).isNull().cast('int')).alias(c) for c in tabela3.columns]).show()

In [ ]:
tabela3 = tabela3.dropna()
tabela3.select([spark_sum(col(c).isNull().cast('int')).alias(c) for c in tabela3.columns]).show()

In [ ]:
print(num_linhas_antes = tabela3.count())
tabela_3 = tabela1.dropDuplicates()
print(num_linhas_depois = tabela_3.count())

In [ ]:
# df['last_processed_time'] = pd.to_datetime(df['last_processed_time'], format='mixed')

# df['first_seen'] = pd.to_datetime(df['first_seen'])
# df.info()

tabela_3 = tabela_3.withColumn('last_processed_time', col('last_processed_time').cast(TimestampType()))
tabela_3 = tabela_3.withColumn('first_seen', col('first_seen').cast(TimestampType()))

In [ ]:
tabela_3.write.csv("linkedin_job_postings_clean.csv", header=True)

### 3º Armazenar em POSTGRESSQL ###